In [8]:
import pandas as pd
import numpy as np

## Reading file 
results = pd.read_csv('submission.csv')
results.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.036132
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.030219
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.022902
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.676110
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.632976


In [9]:
results['label'] = np.where(results['prediction'] > 0.5, 1, 0)
results.head()

,customer_ID,prediction,label
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.036132,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.030219,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.022902,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.676110,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.632976,1


In [10]:
results['label'].value_counts() / results.shape[0]

0    0.797456
1    0.202544
Name: label, dtype: float64

In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# import optuna
# from xgboost import XGBClassifier
from Amex_Metric import amex_metric

import os
import sagemaker

sess = sagemaker.Session()

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'AmericanExpress/Delinquency_Features_Filled.csv'
file_key_2 = 'AmericanExpress/test_delinquency_features.csv'
file_key_3 = 'AmericanExpress/amex_train_payment_spend_final.csv'
file_key_4 = 'AmericanExpress/amex_test_payment_spend_final.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
oscar_data = pd.read_csv(file_content_stream_1, usecols = ['D_44_median', 'D_44_mean', 'D_44_max', 
                                                           'D_75_max', 'D_75_mean', 'D_78_max', 
                                                           'D_78_mean', 'D_78_range', 'D_44_std', 
                                                           'D_75_median', 'D_78_std', 'D_74_mean',
                                                           'D_44_range', 'D_44_min', 'D_84_mean', 
                                                           'D_74_max', 'D_41_range', 'D_75_min', 
                                                           'D_44_IQR', 'D_84_range', 'target', 
                                                           'customer_ID'])

oscar_test = pd.read_csv(file_content_stream_2)

evan_data = pd.read_csv(file_content_stream_3)
evan_data = evan_data.drop(columns = ['target'], axis = 1)

evan_test = pd.read_csv(file_content_stream_4)

## joining files 
data = pd.merge(oscar_data, evan_data, on = 'customer_ID', how = 'left')

In [6]:
oscar_test = pd.read_csv(file_content_stream_4)
oscar_test.head()

ResponseStreamingError: An error occurred while reading from response stream: ("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))

In [2]:
data.head()

,customer_ID,target,D_41_range,D_44_mean,D_44_median,D_44_min,D_44_max,D_44_range,D_44_IQR,D_44_std,...,P_2_median,P_2_correlation,P_3_mean,P_2_sum,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.009171,0.004673,0.004128,0.000630,0.009399,0.008766,0.005070,0.002827,...,0.9385,-0.438767,0.6800,12.140,0.9746,12.670,0.002504,0.001802,0.009280,0.001465
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.008789,0.004311,0.004200,0.000123,0.009872,0.009750,0.004063,0.002970,...,0.9050,-0.854416,0.5670,11.695,0.9756,12.680,0.002622,0.001802,0.008790,0.001465
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.007904,0.013901,0.004570,0.000628,0.128174,0.127563,0.002699,0.034424,...,0.8850,-0.109422,0.6180,11.420,0.9740,12.664,0.002858,0.002329,0.009766,0.003418
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.006695,0.005245,0.005249,0.000058,0.008873,0.008812,0.004976,0.002951,...,0.5980,0.953176,0.6110,7.785,0.9746,12.670,0.002941,0.002403,0.008790,0.004395
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.009468,0.003550,0.002741,0.000342,0.008812,0.008469,0.005122,0.002951,...,0.8794,-0.597527,0.5273,11.590,0.9740,12.664,0.003314,0.002817,0.008790,0.005371


In [4]:
data['target'].value_counts() / data.shape[0]

0    0.741066
1    0.258934
Name: target, dtype: float64

In [3]:
oscar_data.head()

,target,D_41_range,D_44_mean,D_44_median,D_44_min,D_44_max,D_44_range,D_44_IQR,D_44_std,D_74_mean,...,D_75_mean,D_75_median,D_75_min,D_75_max,D_78_mean,D_78_max,D_78_range,D_78_std,D_84_mean,D_84_range
0,0,0.009171,0.004673,0.004128,0.000630,0.009399,0.008766,0.005070,0.002827,0.039520,...,0.036133,0.009895,0.000035,0.076538,0.005215,0.009895,0.009857,0.003622,0.004944,0.008934
1,0,0.008789,0.004311,0.004200,0.000123,0.009872,0.009750,0.004063,0.002970,0.005337,...,0.006470,0.007771,0.000939,0.009598,0.003897,0.008698,0.008537,0.002607,0.004299,0.009529
2,0,0.007904,0.013901,0.004570,0.000628,0.128174,0.127563,0.002699,0.034424,0.019791,...,0.020981,0.006039,0.000628,0.075867,0.005337,0.008568,0.006168,0.002264,0.005062,0.009010
3,0,0.006695,0.005245,0.005249,0.000058,0.008873,0.008812,0.004976,0.002951,0.022476,...,0.020920,0.006660,0.002979,0.075256,0.005760,0.009819,0.008957,0.003164,0.005058,0.008835
4,0,0.009468,0.003550,0.002741,0.000342,0.008812,0.008469,0.005122,0.002951,0.135864,...,0.128052,0.137817,0.067322,0.143066,0.006042,0.009857,0.008568,0.002651,0.005344,0.008972


In [4]:
evan_data.head()

,customer_ID,P_2_mean,P_2_median,P_2_correlation,P_3_mean,P_2_sum,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.9336,0.9385,-0.438767,0.6800,12.140,0.9746,12.670,0.002504,0.001802,0.009280,0.001465,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.9000,0.9050,-0.854416,0.5670,11.695,0.9756,12.680,0.002622,0.001802,0.008790,0.001465,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.8784,0.8850,-0.109422,0.6180,11.420,0.9740,12.664,0.002858,0.002329,0.009766,0.003418,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.5990,0.5980,0.953176,0.6110,7.785,0.9746,12.670,0.002941,0.002403,0.008790,0.004395,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.8916,0.8794,-0.597527,0.5273,11.590,0.9740,12.664,0.003314,0.002817,0.008790,0.005371,0


In [5]:
evan_data.describe()

,P_2_mean,P_2_median,P_2_correlation,P_3_mean,P_2_sum,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr,target
count,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000
mean,0.649243,0.651514,-0.011220,0.600413,7.910978,0.925125,11.150272,0.064724,0.050435,0.178880,0.068770,0.258934
std,0.234731,0.237541,0.619185,0.108319,3.471775,0.138037,2.963924,0.148493,0.118736,0.415668,0.200311,0.438050
min,-0.338400,-0.338400,-1.000000,-1.135000,-3.783000,-2.450000,-17.470000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.471700,0.472000,-0.604590,0.559600,5.230000,0.972700,11.090000,0.002677,0.002140,0.008300,0.003418,0.000000
50%,0.678000,0.681000,-0.024204,0.613300,8.400000,0.973600,12.660000,0.003039,0.002516,0.008790,0.004395,0.000000
75%,0.855000,0.859400,0.583547,0.656200,11.010000,0.974600,12.664000,0.003712,0.003193,0.009766,0.005859,1.000000
max,1.010000,1.010000,1.000000,1.542000,13.090000,2.514000,32.700000,2.423429,1.714000,4.773000,3.580078,1.000000
